In [1]:
from greensight.utils import DIR_DATA
import pandas as pd
import numpy as np
from datetime import datetime
import json
from typing import Union
from pathlib import Path
import re
from tqdm.notebook import tqdm

In [2]:
sentinel_data_path = DIR_DATA / "indices"
assert sentinel_data_path.is_dir()

In [3]:
year_paths = [path for path in sorted(list(sentinel_data_path.iterdir())) if "2015" not in path.stem and "2016" not in path.stem]
year_paths

[PosixPath('/home/finley/Work/RDS/projects/greensight/data/indices/Greenbelts_2017_polygon_bands_and_indices'),
 PosixPath('/home/finley/Work/RDS/projects/greensight/data/indices/Greenbelts_2018_polygon_bands_and_indices'),
 PosixPath('/home/finley/Work/RDS/projects/greensight/data/indices/Greenbelts_2019_polygon_bands_and_indices'),
 PosixPath('/home/finley/Work/RDS/projects/greensight/data/indices/Greenbelts_2020_polygon_bands_and_indices'),
 PosixPath('/home/finley/Work/RDS/projects/greensight/data/indices/Greenbelts_2021_polygon_bands_and_indices'),
 PosixPath('/home/finley/Work/RDS/projects/greensight/data/indices/Greenbelts_2022_polygon_bands_and_indices'),
 PosixPath('/home/finley/Work/RDS/projects/greensight/data/indices/Greenbelts_2023_polygon_bands_and_indices'),
 PosixPath('/home/finley/Work/RDS/projects/greensight/data/indices/Greenbelts_2024_polygon_bands_and_indices')]

In [4]:
def load_sentinel_two_index_data_from_csv(path: Union[str, Path]) -> pd.DataFrame:

    """""
    loads sentinel two data from file path and process into dataframe 
    """
    path = Path(path)
    assert path.is_file()

    df = pd.read_csv(path)

    # Extract the year using regex
    match = re.search(r"\d{4}", str(path))
    year = int(match.group(0) if match else None)

    df = df.drop(columns=["system:index", ".geo"])
    # set index
    df = df.set_index("LAD_CD")

    # check for duplicates
    assert np.unique(df.columns).shape == df.columns.shape
    index_inds = list(set(
        sorted([i.split("_")[1] for i in df.columns.unique() if i.split("_")[0].isnumeric()])
    ))
    month_inds = set(
        [i.split("_")[0] for i in df.columns.unique() if i.split("_")[0].isnumeric()]
    )

    months = []
    inds = []
    for month in month_inds:
        # generate desired columns
        required_cols = [month + "_" + band for band in index_inds]

        cols = [col for col in required_cols if col in df.columns] 
        df_month = df[cols].copy()

        # convert from a DataFrame of rows: shapes, columns: bands for a single month to a single row of rows: month, columns: (shape, band)
        row_month = df_month.stack().to_frame().T

                # create multi-index for the columns (shape, band)
        new_cols = [(a, b.split("_")[1]) for a, b in row_month.columns]
        row_month.columns = pd.MultiIndex.from_tuples(new_cols)

        # add to stack
        months.append(row_month)
        # add month name to index.
        inds.append(month)

    # combine rows
    df_month = pd.concat(months, axis=0)

    # fix index to month value
    df_month.index = np.array(inds).astype(int) + 1

    # format index
    df_month = df_month.sort_index()
    df_month.index.name = "date"
    df_month.index = [datetime(year, int(month), 1) for month in df_month.index]
    df_month.columns.names = ("shape", "band")

    assert df_month.shape == (len(month_inds), len(index_inds)*df.shape[0])

    # add greenbelt information from json dict.
    lookup_path = DIR_DATA / "id_lookup/id_lookup.json"
    with open(lookup_path, "r") as in_file:
        D_lookup = json.load(in_file)
    greenbelts = [D_lookup[code]["GB_Name"] for code, _ in df_month.columns]

    # add greenbelts to column MultiIndex
    df_month.columns = pd.MultiIndex.from_tuples(
        [(gb, *cols) for gb, cols in zip(greenbelts, df_month.columns)]
    )
    df_month.columns.names = ("greenbelt", "shape", "band")

    return df_month

In [5]:
df_all = []
for year_path in tqdm(year_paths):
    year_files = sorted([file for file in year_path.iterdir()])

    df_year = []
    for file in tqdm(year_files):
        try: 
            out = load_sentinel_two_index_data_from_csv(file)
            if out is not None:
                df_year.append(out)
        except:
            print(f"Failed file {file}")
    
    if df_year != []:
        df_year = pd.concat(df_year, axis=1)

        df_year = df_year.loc[:, ~df_year.columns.duplicated()]
        df_all.append(df_year)


  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/95 [00:00<?, ?it/s]

Failed file /home/finley/Work/RDS/projects/greensight/data/indices/Greenbelts_2017_polygon_bands_and_indices/feature_vectors_S2_0000000185_0000000185_S2_mean.csv


  0%|          | 0/93 [00:00<?, ?it/s]

  0%|          | 0/93 [00:00<?, ?it/s]

...

Failed file /home/finley/Work/RDS/projects/greensight/data/indices/Greenbelts_2022_polygon_bands_and_indices/feature_vectors_S2_0000000071_0000000073_S2_mean.csv
Failed file /home/finley/Work/RDS/projects/greensight/data/indices/Greenbelts_2022_polygon_bands_and_indices/feature_vectors_S2_0000000086_0000000088_S2_mean.csv
Failed file /home/finley/Work/RDS/projects/greensight/data/indices/Greenbelts_2022_polygon_bands_and_indices/feature_vectors_S2_0000000185_0000000185_S2_mean.csv


  0%|          | 0/93 [00:00<?, ?it/s]

Failed file /home/finley/Work/RDS/projects/greensight/data/indices/Greenbelts_2023_polygon_bands_and_indices/feature_vectors_S2_0000000072_0000000073_S2_mean.csv


  0%|          | 0/93 [00:00<?, ?it/s]

Failed file /home/finley/Work/RDS/projects/greensight/data/indices/Greenbelts_2024_polygon_bands_and_indices/feature_vectors_S2_0000000072_0000000073_S2_mean.csv


In [6]:
# remove duplicate column indices
for i, df in enumerate(df_all):
    df_all[i] = df.loc[:, ~df.columns.duplicated()]

In [7]:
df_out = pd.concat(df_all, axis=0)

In [8]:
df_out

greenbelt  Bath and Bristol                                             \
shape             E06000022                                              
band                   ndwi      ndti          dvi      ndpi      arvi   
2017-04-01        -0.698364 -0.337793  3575.746062  0.337793  0.662791   
2017-05-01        -0.701280 -0.359582  3780.281624  0.359582  0.675456   
2017-06-01        -0.670956 -0.295201  3675.092187  0.295201  0.601951   
2017-07-01        -0.614131 -0.251190  3373.880123  0.251190  0.582856   
2017-08-01              NaN       NaN          NaN       NaN       NaN   
...                     ...       ...          ...       ...       ...   
2024-08-01        -0.602442 -0.211111  2765.168691  0.211111  0.495025   
2024-09-01        -0.605956 -0.213965  2755.027028  0.213965  0.516476   
2024-10-01        -0.615078 -0.259195  2713.886433  0.259195  0.561049   
2024-11-01        -0.599835 -0.240204  2331.800852  0.240204  0.532243   
2024-12-01        -0.597630 -0.239063  2156.893776  0.239063  0.521653   

greenbelt                                                        ...  \
shape                                                            ...   
band               wdvi     pssra      gemi      ndvi      savi  ...   
2017-04-01  3957.022082 -0.337793 -0.479395  0.774909  1.162225  ...   
2017-05-01  4156.170441 -0.359582 -0.664718  0.782279  1.173305  ...   
2017-06-01  4171.652016 -0.295201 -0.933618  0.733432  1.100133  ...   
2017-07-01  3808.096822 -0.251190 -3.865655  0.685344  1.028251  ...   
2017-08-01          NaN       NaN       NaN       NaN       NaN  ...   
...                 ...       ...       ...       ...       ...  ...   
2024-08-01  3377.440772 -0.211111 -0.401834  0.650139  0.975088  ...   
2024-09-01  3309.601806 -0.213965 -0.409297  0.665505  0.998129  ...   
2024-10-01  3144.078124 -0.259195 -0.406479  0.693836  1.040592  ...   
2024-11-01  2722.982283 -0.240204 -0.501182  0.665267  0.997745  ...   
2024-12-01  2543.127812 -0.239063 -0.576855  0.657353  0.985877  ...   

greenbelt       York                                                      \
shape      E06000014                                                       
band           ndi45     tndvi      ipvi       mtci         ri     ireci   
2017-04-01  0.434996  0.419851  0.328669   6.601490   2.472169  0.017848   
2017-05-01  0.699359  0.754130  0.671939  12.688500  30.108144 -0.239381   
2017-06-01       NaN       NaN       NaN        NaN        NaN       NaN   
2017-07-01       NaN       NaN       NaN        NaN        NaN       NaN   
2017-08-01  0.604316  0.621902  0.527854   6.274773   5.812412 -0.067694   
...              ...       ...       ...        ...        ...       ...   
2024-08-01  0.544992  0.549131  0.459687   8.964998   5.281019 -0.049443   
2024-09-01  0.562622  0.564775  0.480541   7.996962   6.476226 -0.066050   
2024-10-01  0.560214  0.588430  0.493619   7.857577   5.811782 -0.084636   
2024-11-01  0.598243  0.618075  0.523736   6.973810   6.902465 -0.062675   
2024-12-01  0.548785  0.590795  0.490100   7.521339   5.363163 -0.094763   

greenbelt                      Burton-upon-Trent and Swadlincote        
shape                                                  E07000193        
band           s2rep        ci                            msavi2 msavi  
2017-04-01  0.128857  0.419859                               NaN   NaN  
2017-05-01  0.314883  0.754145                               NaN   NaN  
2017-06-01       NaN       NaN                               NaN   NaN  
2017-07-01       NaN       NaN                               NaN   NaN  
2017-08-01  0.226076  0.621914                               NaN   NaN  
...              ...       ...                               ...   ...  
2024-08-01  0.155171  0.549143                               NaN   NaN  
2024-09-01  0.086108  0.564789                               NaN   NaN  
2024-10-01  0.154923  0.588446                               NaN   NaN  
2024-11-01  0

In [9]:
# clean up greenbelt place names
df_out.columns = pd.MultiIndex.from_tuples([(i[0].replace("-", " ").replace(",", "") , i[1], i[2]) for i in df_out.columns])
df_out.columns.names = ["greenbelt", "shape", "band"]
df_out.head()

greenbelt  Bath and Bristol                                             \
shape             E06000022                                              
band                   ndwi      ndti          dvi      ndpi      arvi   
2017-04-01        -0.698364 -0.337793  3575.746062  0.337793  0.662791   
2017-05-01        -0.701280 -0.359582  3780.281624  0.359582  0.675456   
2017-06-01        -0.670956 -0.295201  3675.092187  0.295201  0.601951   
2017-07-01        -0.614131 -0.251190  3373.880123  0.251190  0.582856   
2017-08-01              NaN       NaN          NaN       NaN       NaN   

greenbelt                                                        ...  \
shape                                                            ...   
band               wdvi     pssra      gemi      ndvi      savi  ...   
2017-04-01  3957.022082 -0.337793 -0.479395  0.774909  1.162225  ...   
2017-05-01  4156.170441 -0.359582 -0.664718  0.782279  1.173305  ...   
2017-06-01  4171.652016 -0.295201 -0.933618  0.733432  1.100133  ...   
2017-07-01  3808.096822 -0.251190 -3.865655  0.685344  1.028251  ...   
2017-08-01          NaN       NaN       NaN       NaN       NaN  ...   

greenbelt       York                                                      \
shape      E06000014                                                       
band           ndi45     tndvi      ipvi       mtci         ri     ireci   
2017-04-01  0.434996  0.419851  0.328669   6.601490   2.472169  0.017848   
2017-05-01  0.699359  0.754130  0.671939  12.688500  30.108144 -0.239381   
2017-06-01       NaN       NaN       NaN        NaN        NaN       NaN   
2017-07-01       NaN       NaN       NaN        NaN        NaN       NaN   
2017-08-01  0.604316  0.621902  0.527854   6.274773   5.812412 -0.067694   

greenbelt                      Burton upon Trent and Swadlincote        
shape                                                  E07000193        
band           s2rep        ci                            msavi2 msavi  
2017-04-01  0.128857  0.419859                               NaN   NaN  
2017-05-01  0.314883  0.754145                               NaN   NaN  
2017-06-01       NaN       NaN                               NaN   NaN  
2017-07-01       NaN       NaN                               NaN   NaN  
2017-08-01  0.226076  0.621914                               NaN   NaN  

[5 rows x 5400 columns]

In [10]:
output_path = DIR_DATA / "processed_data/sentinel_two_indices.hdf"
df_out.to_parquet(output_path)

In [11]:
df_in = pd.read_parquet(output_path)

In [12]:
np.testing.assert_array_equal(df_in.index.values, df_out.index.values)
np.testing.assert_array_equal(df_in.columns.values, df_out.columns.values)
np.testing.assert_array_equal(df_in.values, df_out.values)